# PRMT-2498 - 1 Jan to 7 Jan - Transfer level report and Error Combination Report

## Context
Produce a transfer level report for failures, and the error combination table using:
- Date range: Jan 1st - Jan 7th (7 days)
- Cut off of 1 day

In [1]:
import pandas as pd 
import numpy as np
import paths, data
from sklearn.preprocessing import MultiLabelBinarizer
from datetime import datetime
from data.practice_metadata import read_asid_metadata

In [2]:
error_code_lookup_file = pd.read_csv(data.gp2gp_response_codes.path)
error_code_lookup = error_code_lookup_file.set_index("ErrorCode")["ErrorName"]

In [3]:
asid_lookup = read_asid_metadata("prm-gp2gp-ods-metadata-dev", "v2/2021/12/organisationMetadata.json")
transfer_file_location = "s3://prm-gp2gp-notebook-data-dev/PRMT-2498-first-week-jan/transfer-data/v6/2022/1/2022-1-transfers.parquet"

transfers_raw = pd.read_parquet(transfer_file_location)
transfers = transfers_raw\
    .join(asid_lookup.add_prefix("requesting_"), on="requesting_practice_asid", how="left")\
    .join(asid_lookup.add_prefix("sending_"), on="sending_practice_asid", how="left")\

transfers["status"] = transfers["status"].str.replace("_", " ").str.capitalize()

len(transfers)

39979

In [4]:
# Check that transfers are all between 1st Jan to 7th Jan
first_half_date_filter_bool_start = transfers["date_requested"] >= datetime(2022, 1, 1)
first_half_date_filter_bool_end = transfers["date_requested"] < datetime(2022, 1, 8)
transfers_within_date_range = transfers[first_half_date_filter_bool_start & first_half_date_filter_bool_end]
len(transfers_within_date_range)

39979

## Generating the transfer level report

In [5]:
# Look for transfers that have status Technical Failure or Unclassified Failure
failed_transfers_bool = (transfers_within_date_range["status"] == "Technical failure")| (transfers_within_date_range["status"] == "Unclassified failure")
failed_transfers = transfers_within_date_range[failed_transfers_bool]
len(failed_transfers)

1525

In [6]:
# Remove Nan and duplicate occurrences in error codes, and sort them
def filter_error_codes(error_codes_column):
    return error_codes_column.apply(lambda error_codes: [error_code for error_code in sorted(set(error_codes)) if not np.isnan(error_code)])

# This is important otherwise notebook will complain about mutating original list/dataset in function calls below
failed_transfers = failed_transfers.copy()

failed_transfers["unique_sender_error_codes"] = filter_error_codes(failed_transfers["sender_error_codes"])
failed_transfers["unique_final_error_codes"] = filter_error_codes(failed_transfers["final_error_codes"])
failed_transfers["unique_intermediate_error_codes"] = filter_error_codes(failed_transfers["intermediate_error_codes"])
failed_transfers.head(1)

,conversation_id,sla_duration,requesting_practice_asid,sending_practice_asid,requesting_supplier,sending_supplier,sender_error_codes,final_error_codes,intermediate_error_codes,status,...,requesting_practice_name,requesting_ccg_ods_code,requesting_ccg_name,sending_practice_ods_code,sending_practice_name,sending_ccg_ods_code,sending_ccg_name,unique_sender_error_codes,unique_final_error_codes,unique_intermediate_error_codes
6,7AF0F6F0-6AE5-11EC-BA8B-3F5E6F335CD5,NaN,118930647011,469194707045,SystmOne,EMIS,[20.0],[],[],Technical failure,...,GP AT HAND,W2U3Z,NHS NORTH WEST LONDON CCG,G83021,VANBRUGH GROUP PRACTICE,72Q,NHS SOUTH EAST LONDON CCG,[20.0],[],[]


In [7]:
error_code_lookup_file = pd.read_csv(data.gp2gp_response_codes.path)
error_code_lookup = error_code_lookup_file.set_index("ErrorCode")["ErrorName"]

In [8]:
# Add descriptions from lookup to error code columns
def convert_error_list_to_formatted_string_list_with_error_lookup(error_code_list):
    return ", ".join([str(error_code.astype(int))+" - "+str(error_code_lookup[error_code]) for error_code in error_code_list])

failed_transfers["unique_sender_error_codes_with_description"] = failed_transfers["unique_sender_error_codes"].apply(convert_error_list_to_formatted_string_list_with_error_lookup)
failed_transfers["unique_final_error_codes_with_description"] = failed_transfers["unique_final_error_codes"].apply(convert_error_list_to_formatted_string_list_with_error_lookup)
failed_transfers["unique_intermediate_error_codes_with_description"] = failed_transfers["unique_intermediate_error_codes"].apply(convert_error_list_to_formatted_string_list_with_error_lookup)

In [9]:
# Prune columns we want to keep for the report
columns_to_keep = [
    'sending_practice_ods_code',
    'sending_supplier',
    'requesting_practice_ods_code',    
    'requesting_supplier',
    'conversation_id', 
    'date_requested', 
    'status',
    'failure_reason', 
    'unique_sender_error_codes_with_description', 
    'unique_final_error_codes_with_description',
    'unique_intermediate_error_codes_with_description']

failed_transfers_with_filtered_and_renamed_columns = failed_transfers[columns_to_keep]

failed_transfers_with_filtered_and_renamed_columns=failed_transfers_with_filtered_and_renamed_columns.rename({
    "sending_practice_ods_code":"Sending Practice ODS",
    "sending_supplier":"Sending Supplier",
    "requesting_practice_ods_code":"Requesting Practice ODS",
    "requesting_supplier":"Requesting Supplier",
    "conversation_id":"Conversation ID",
    "date_requested":"Date Requested",
    "status":"Status",
    "failure_reason":"Failure Reason",
    "unique_sender_error_codes_with_description":"Unique Sender Errors",
    "unique_final_error_codes_with_description":"Unique Final Errors",
    "unique_intermediate_error_codes_with_description":"Unique Intermediate Errors"
    },axis=1)

failed_transfers_with_filtered_and_renamed_columns.head(2)

,Sending Practice ODS,Sending Supplier,Requesting Practice ODS,Requesting Supplier,Conversation ID,Date Requested,Status,Failure Reason,Unique Sender Errors,Unique Final Errors,Unique Intermediate Errors
6,G83021,EMIS,E85124,SystmOne,7AF0F6F0-6AE5-11EC-BA8B-3F5E6F335CD5,2022-01-01 09:29:31.566,Technical failure,Core extract not sent,20 - Spine error,,
50,G83012,EMIS,G83067,EMIS,678AE9B4-7D44-4C75-870C-061C936BB51D,2022-01-01 17:32:56.804,Technical failure,Request not acknowledged,,,


## Unique error codes count

In [10]:
data = {
    'unique_sender_error_codes_count': failed_transfers.explode("unique_sender_error_codes").value_counts("unique_sender_error_codes"),
    'unique_final_error_codes_count': failed_transfers.explode("unique_final_error_codes").value_counts("unique_final_error_codes"),
    'unique_intermediate_error_codes_count': failed_transfers.explode("unique_intermediate_error_codes").value_counts("unique_intermediate_error_codes")
    }
df = pd.DataFrame(data=data)

print("18 November - 2 December after fix [total transfers:"+str(len(transfers_within_date_range))+", total failed transfers (technical + unclassified): "+str(len(failed_transfers))+"]")
df.sort_values(by=["unique_sender_error_codes_count", "unique_final_error_codes_count", "unique_intermediate_error_codes_count"], ascending=False)

18 November - 2 December after fix [total transfers:39979, total failed transfers (technical + unclassified): 1525]


,unique_sender_error_codes_count,unique_final_error_codes_count,unique_intermediate_error_codes_count
10.0,315.0,NaN,NaN
30.0,288.0,109.0,NaN
20.0,277.0,NaN,NaN
6.0,26.0,NaN,NaN
24.0,6.0,NaN,NaN
19.0,4.0,NaN,NaN
14.0,2.0,NaN,NaN
23.0,2.0,NaN,NaN
99.0,1.0,31.0,NaN
12.0,NaN,127.0,NaN


In [11]:
df.sum(axis=0)

unique_sender_error_codes_count          921.0
unique_final_error_codes_count           378.0
unique_intermediate_error_codes_count     70.0
dtype: float64

## Generating High level table

In [12]:
# Format the errors to be readable strings
transfers_within_date_range["unique_sender_error_codes"] = filter_error_codes(transfers_within_date_range["sender_error_codes"])
transfers_within_date_range["unique_final_error_codes"] = filter_error_codes(transfers_within_date_range["final_error_codes"])
transfers_within_date_range["unique_intermediate_error_codes"] = filter_error_codes(transfers_within_date_range["intermediate_error_codes"])

transfers_within_date_range["unique_sender_error_codes_with_description"] = transfers_within_date_range["unique_sender_error_codes"].apply(convert_error_list_to_formatted_string_list_with_error_lookup)
transfers_within_date_range["unique_final_error_codes_with_description"] = transfers_within_date_range["unique_final_error_codes"].apply(convert_error_list_to_formatted_string_list_with_error_lookup)
transfers_within_date_range["unique_intermediate_error_codes_with_description"] = transfers_within_date_range["unique_intermediate_error_codes"].apply(convert_error_list_to_formatted_string_list_with_error_lookup)
transfers_within_date_range.head(1)

,conversation_id,sla_duration,requesting_practice_asid,sending_practice_asid,requesting_supplier,sending_supplier,sender_error_codes,final_error_codes,intermediate_error_codes,status,...,sending_practice_ods_code,sending_practice_name,sending_ccg_ods_code,sending_ccg_name,unique_sender_error_codes,unique_final_error_codes,unique_intermediate_error_codes,unique_sender_error_codes_with_description,unique_final_error_codes_with_description,unique_intermediate_error_codes_with_description
0,CC6BDD63-C6A3-4534-AF12-849AA740FF6D,NaN,020881902017,113325050012,EMIS,SystmOne,[nan],[],[],Process failure,...,M85028,LORDSWOOD HOUSE GROUP MEDICAL PRACTICE,15E,NHS BIRMINGHAM AND SOLIHULL CCG,[],[],[],,,


In [13]:
def generate_high_level_table(transfers_sample):
    # Create High level table
    high_level_table=transfers_sample.fillna("N/A").groupby(["requesting_supplier","sending_supplier","status","failure_reason", "unique_final_error_codes_with_description", "unique_sender_error_codes_with_description", "unique_intermediate_error_codes_with_description"]).agg({"conversation_id":"count"})
    high_level_table=high_level_table.rename({"conversation_id":"number of transfers"},axis=1).reset_index()

    # Count % of transfers
    total_number_transfers = transfers_sample.shape[0]
    high_level_table["% of transfers"]=(high_level_table["number of transfers"]/total_number_transfers).multiply(100)
    
    # Count % of technical failures
    technical_failed_transfers_bool = transfers_sample["status"] == "Technical failure"
    technical_failed_transfers = transfers_sample[technical_failed_transfers_bool]
    total_number_technical_failed_transfers = technical_failed_transfers.shape[0]
    technical_failed_transfers_table_bool = high_level_table["status"] == "Technical failure"
    high_level_table.loc[technical_failed_transfers_table_bool, "% of technical failures"]=(high_level_table.loc[technical_failed_transfers_table_bool, "number of transfers"]/total_number_technical_failed_transfers).multiply(100)
    
    # Count by supplier pathway
    supplier_pathway_counts = transfers_sample.fillna("Unknown").groupby(by=["sending_supplier", "requesting_supplier"]).agg({"conversation_id": "count"})["conversation_id"]
    high_level_table["% of supplier pathway"]=high_level_table.apply(lambda row: row["number of transfers"]/supplier_pathway_counts.loc[(row["sending_supplier"],row["requesting_supplier"])],axis=1).multiply(100)

    # Select and re-order table
    grouping_columns_order=["requesting_supplier","sending_supplier","status","failure_reason", "unique_final_error_codes_with_description", "unique_sender_error_codes_with_description", "unique_intermediate_error_codes_with_description"]
    counting_columns_order=["number of transfers","% of transfers","% of technical failures","% of supplier pathway"]
    high_level_table=high_level_table[grouping_columns_order+counting_columns_order].sort_values(by="number of transfers",ascending=False)
    
    # Rename the columns
    high_level_table=high_level_table.rename({
        "sending_supplier":"sending supplier",
        "requesting_supplier":"requesting supplier",
        "status":"status",
        "failure_reason":"failure reason",
        "unique_final_error_codes_with_description":"unique final errors",
        "unique_sender_error_codes_with_description":"unique sender errors",
        "unique_intermediate_error_codes_with_description":"unique intermediate errors"
        },axis=1)
    
    # Replace all N/A with empty strings primarily for error columns
    high_level_table=high_level_table.replace("N/A", "")
    
    return high_level_table

In [14]:
with pd.ExcelWriter("PRMT-2498-Error-code-combination-and-transfer-level-table-1-Jan-7-Jan.xlsx") as writer:
     generate_high_level_table(transfers_within_date_range).to_excel(writer, sheet_name="Error Code Combination Table",index=False)
     failed_transfers_with_filtered_and_renamed_columns.to_excel(writer, sheet_name="Transfer level failures",index=False)